In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras_preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, ReLU, LeakyReLU, SimpleRNN
from keras.optimizers import SGD
from konlpy.tag import Okt 

Using TensorFlow backend.


In [2]:
csv_data = pd.read_csv('movie.csv', encoding='utf8')
csv_data = csv_data[csv_data.평점 != 5]
csv_data = csv_data[csv_data.평점 != 6]
csv_data = csv_data[csv_data.평점 != 7]
csv_data = csv_data[csv_data.평점 != 8]
csv_data['평점'] = csv_data['평점'].apply(lambda x: 1 if x>=9 else 0)
csv_data.reset_index(drop=True, inplace=True)

In [3]:
okt = Okt()
token = [okt.pos(csv_data['평가'][i], stem=True, norm=True) for i in range(len(csv_data))]

/Users/lunab/anaconda3/lib/python3.7/site-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [4]:
tokenizer = Tokenizer()

max_len = 0
x_data = []
for t in token:
    tmp = [[i[0]+'/'+i[1]] for i in np.array(t)]
    x_data.append(tmp)
    if max_len < len(tmp):
        max_len = len(tmp)
    tokenizer.fit_on_texts(tmp)
print(len(tokenizer.word_index))

10169


In [5]:
x_data_seq = []
for x in x_data:
    x_data_seq.append(tokenizer.texts_to_sequences(x))

In [6]:
x_pad_data = sequence.pad_sequences(x_data_seq, maxlen=max_len)

In [7]:
# x_train = np.asarray(x_pad_data, np.float32)/len(tokenizer.word_index)
x_train = x_pad_data

In [8]:
# y_train = []
# for d in csv_data['평점']:
#     if d == 0:
#         y_train.append([1,0])
#     else:
#         y_train.append([0,1])
        
y_train = []
for d in csv_data['평점']:
    if d == 0:
        y_train.append([1,0])
    else:
        y_train.append([0,1])

In [12]:
x_train_ = x_train.reshape((-1,max_len))
y_train_ = np.array(y_train).reshape((-1,2))
print(x_train_.shape)
print(y_train_.shape)

(7667, 67)
(7667, 2)


In [13]:
np.random.seed(0)
model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, len(y_train_), input_length=max_len))
model.add(LSTM(68, input_shape=(max_len,1)))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
history = model.fit(x_train_, y_train_, epochs=3, verbose=1, validation_split=0.2)

W0717 20:02:18.470543 4519339456 deprecation_wrapper.py:119] From /Users/lunab/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 6133 samples, validate on 1534 samples
Epoch 1/3
1632/6133 [======>.......................] - ETA: 3:27 - loss: 0.4602 - acc: 0.8061

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Loss")
plt.show()
plt.plot(history.history["acc"])
plt.plot(history.history["val_acc"])
plt.title("Acc")
plt.show()

In [ ]:
predictions = model.predict(x_train_)
for i in range(len(predictions)):
    if np.argmax(predictions[i]) == 1:
        print(1, end=' ')
    else:
        print(0, end=' ')
    print(csv_data['평가'][i])

In [ ]:
def prediction(text=[1]):
    t = okt.pos(text, stem=True, norm=True)
    s = tokenizer.texts_to_sequences([[i[0]+'/'+i[1]] for i in np.array(t)])
    p = sequence.pad_sequences([s], maxlen=max_len)
    x = np.array(p).reshape((1,-1))
    y = model.predict(x)
    yp = np.argmax(y)
    return y, yp

text = '아 진짜 짜증나네'
y, yp = prediction(text)
print(y, yp)
if yp == 0:
    print('"%s"는\n%f의 확률로 부정적입니다.'%(text, y[0,0]))
else:
    print('"%s"는\n%f의 확률로 긍정적입니다.'%(text, y[0,1]))
        

In [ ]:
# 모델 저장
model_json = model.to_json()
with open("model.json", "w") as json_file : 
    json_file.write(model_json)
    
model.save_weights("model.h5")
print("save")

In [ ]:
# 모델 로드
from keras.models import model_from_json 

json_file = open("model.json", "r")
loaded_model_json = json_file.read() 
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("model.h5")
print("Loaded model from disk")

# compile
# loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# loaded_model_history = model.fit(x_train_, y_train_, epochs=20, verbose=1)

In [15]:
print(1)

1
